# Character-Consistent Storyboard Generation
In this hands-on session, we'll incorporate character consistency techniques to create cohesive storyboards for visual storytelling and animation development using Amazon Bedrock.

## Workshop Objectives
By the end of this workshop, you will:
1. Understand how to use generative AI to create structured story scenes from a simple concept.
2. Learn how to utilize character and style consistency techniques to create consistent storyboard panels using Nova Canvas.
3. Use the generated panel images as references to create animated storyboards using Nova Reel.
4. Develop a complete storyboard workflow from concept to visualization.


## Introduction to Storyboard Generation

Storyboarding is a critical step in the visual development process for animation, film, and other visual media. It helps visualize the narrative flow, camera angles, character positioning, and scene transitions before committing to full production.

In this notebook, we'll use a combination of generative AI models to create a complete storyboarding workflow:

1. **Claude 3 Sonnet** - For generating structured story scenes and refining image prompts
2. **Nova Canvas** - For creating consistent storyboard panels
3. **Nova Reel** - For animating selected panels into short video clips

To begin, run the cell below to set up the necessary dependencies and create instances of the Bedrock clients we'll use throughout this workshop.


In [ ]:
%pip install -r requirements.txt


<div class="alert alert-block alert-warning">
<strong>⚠️ Important:</strong> Make sure your IAM role has the necessary permissions to invoke Bedrock models and access S3 for storing generated assets.
</div>

### IAM Permissions Requirements

To run this notebook successfully, your IAM role needs the following permissions:
- `bedrock:InvokeModel` - For calling the Bedrock models
- `bedrock:GetAsyncInvoke` - For handling asynchronous model invocations

You can use the following policy to grant these permissions:

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:GetAsyncInvoke"
            ],
            "Resource": "*"
        }
    ]
}
```


In [ ]:
import boto3
import json
import time
import sagemaker

from helpers.bedrock_helpers import (generate_text, generate_images, get_random_seed, 
                                     generate_videos)
from helpers.display_helpers import (display_story_table, display_prompt_table, 
                                     display_storyboard, display_video, display_text)
from helpers.prompt_helpers import (system_prompts, get_character_descriptions, 
                                    get_style_prompt)

session = boto3.Session(region_name="us-east-1")

bedrock_client = session.client("bedrock-runtime")
s3_client = session.client("s3")

session = sagemaker.Session(boto_session=session)
default_bucket = session.default_bucket()

# foundation model used to develop the storyboard
text_generation_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
image_generation_model_id = "amazon.nova-canvas-v1:0"
video_generation_model_id = "amazon.nova-reel-v1:0"

## Step 1: Story Generation with Claude

The first step in our storyboarding process is to generate a structured story with well-defined scenes. We'll use Claude 3 Sonnet to create a story based on a simple idea or concept.

In the cells below, we'll:
1. Define our story parameters (scene count, genre, and basic idea)
2. Optionally specify existing characters to include
3. Generate a structured story with scenes, descriptions, and imagery guidance

Let's start by defining our story parameters:

In [ ]:
scene_count = 3
genre = "Family"
idea = "a small robot is lost in a library and needs to read some books to find its way out"
characters = []

Now, let's create a prompt for Claude to generate our story structure. The prompt instructs Claude to act as an award-winning director and create a specified number of scenes based on our story idea.

In [ ]:
user_prompt = f"""
        Human: You are an award wining director, create {scene_count} scenes from the <StoryIdea> for storyboarding. The genre is {genre}.
            <StoryIdea>
            {idea}
            </StoryIdea>
        Use the specified characters in the story.  You can add more if needed.
            <Characters>
            {json.dumps(characters)}
            </Characters>
"""

Let's generate the story structure using Claude and display the results in a table format:

In [ ]:
story = generate_text(bedrock_client, text_generation_model_id, user_prompt, system_prompts["story"])
display_story_table(story)

## Step 2: Image Prompt Generation

Now that we have our story structure, we need to create effective prompts for Nova Canvas to generate our storyboard panels. Good prompts are crucial for getting high-quality, consistent images that accurately represent our scenes.

In this step, we'll:
1. Process each scene to create a base image prompt
2. Use Claude to refine and condition these prompts for optimal results
3. Apply a consistent visual style to all prompts

Let's generate our image prompts:

In [ ]:
style = "3D"
throttle_pause_time = 0

image_prompts = {}
for scene in story.get("scenes"):
    print("Processing scene {}/{} ...".format(scene["scene_id"]+1, story.get("scene_count")))
    base_image_prompt = f"""Generate a storyboard image for the following scene:
            Scene decription:
            {scene["description"]}
            Scene imagary:
            {scene["imagery"]}
            Character descriptions:
            {get_character_descriptions(scene)}
            """
    conditioned_image_prompt = generate_text(bedrock_client, text_generation_model_id, base_image_prompt, system_prompts["image"])
    time.sleep(throttle_pause_time)
    styled_image_prompt = generate_text(bedrock_client, text_generation_model_id, f"Image generation prompt: {conditioned_image_prompt}", get_style_prompt(style))
    image_prompts[scene["scene_id"]] = styled_image_prompt["prompt"]
    time.sleep(throttle_pause_time)

display_prompt_table(story_data=story, image_prompts=image_prompts)

## Step 3: Storyboard Panel Generation

With our refined image prompts ready, we can now generate storyboard panels using Nova Canvas. To ensure we have options for each scene, we'll generate multiple variations per scene.

In this step, we'll:
1. Generate multiple image variations for each scene
2. Use a consistent seed value to maintain visual coherence
3. Apply negative prompting to avoid unwanted elements

Let's generate our storyboard panels:

In [ ]:
images_per_scene = 3
negative_prompt = "ugly, yellow, red, green, blue, pink, orange, brown, old, dirty, colorful"
seed = get_random_seed()

storyboard_images = {}
for scene in story.get("scenes"):
    scene_id = scene["scene_id"]
    print("Processing scene {}/{} ...".format(scene_id+1, story.get("scene_count")))
    prompt = image_prompts[scene_id]
    images = generate_images(bedrock_client, image_generation_model_id, prompt, negative_prompt, seed=seed, image_count=images_per_scene)
    storyboard_images[scene_id] = images
display_storyboard(storyboard_images, story)

## Step 4: Video Generation from Storyboard Panel

As a final step, we can bring one of our storyboard panels to life by generating a short video clip using Nova Reel. This can help visualize how the scene might look in motion.

In this step, we'll:
1. Select the first image from the first scene
2. Generate a video prompt based on the image prompt
3. Create a short video clip using Nova Reel

Let's generate our video:

In [ ]:
video_prompt = generate_text(bedrock_client, text_generation_model_id, image_prompts[0], system_prompts["video"]).get("prompt")
display_text(f"> Video prompt: {video_prompt}")

s3_location = generate_videos(bedrock_client, video_generation_model_id, video_prompt, storyboard_images[0][0], default_bucket)


Finally, let's download and display the generated video:

In [ ]:
video_output = "output.mp4"
s3_prefix = s3_location.split("/")[-1]
s3_client.download_file(default_bucket, f"{s3_prefix}/{video_output}", video_output)

# Display the video in the notebook
display_video(video_output)

## Conclusion

In this workshop, you've learned how to create a complete storyboarding workflow using Amazon Bedrock's generative AI capabilities:

1. Using Claude to generate structured story scenes
2. Creating optimized image prompts for Nova Canvas
3. Generating consistent storyboard panels with visual coherence
4. Bringing storyboard panels to life with Nova Reel

This workflow can significantly accelerate the pre-production process for animation, film, and other visual media projects, allowing creators to quickly visualize concepts and iterate on ideas before committing to full production.